In [1]:
import numpy as np
import pandas as pd
import csv

In [2]:
from __future__ import absolute_import, division
import csv
import random as rn
import os
import sys
os.environ['PYTHONHASHSEED'] = '0'
# os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   # see issue #152
# os.environ["CUDA_VISIBLE_DEVICES"] = ""
# import numpy
np.random.seed(1)
import pandas as pd
# import tensorflow
# tensorflow.set_random_seed(2)
rn.seed(1)
# session_conf = tensorflow.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
os.environ['KERAS_BACKEND'] = 'theano'
from keras import backend as K
# sess = tensorflow.Session(graph=tensorflow.get_default_graph(), config=session_conf)
# K.set_session(sess)

/home/prabod/anaconda2/lib/python2.7/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using Theano backend.
Using cuDNN version 7005 on context None
Mapped name None to device cuda: GeForce GTX 950M (0000:01:00.0)


In [3]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.metrics import classification_report

In [4]:
trainData = pd.read_table("preprocessed/2018-E-c-En-train.txt",sep="\t", index_col=0, quoting=csv.QUOTE_NONE)
testData = pd.read_table("preprocessed/2018-E-c-En-test.txt", sep="\t", index_col=0, quoting=csv.QUOTE_NONE)
devData = pd.read_table("preprocessed/2018-E-c-En-dev.txt", sep="\t", index_col=0, quoting=csv.QUOTE_NONE)
gold = pd.read_table("2018-E-c-En-test-gold.txt", sep="\t", index_col=0, quoting=csv.QUOTE_NONE)

In [ ]:
# trainData = trainData.drop(['anticipation','trust','surprise','pessimism'],axis=1)
# testData = testData.drop(['anticipation','trust','surprise','pessimism'],axis=1)
# devData = devData.drop(['anticipation','trust','surprise','pessimism'],axis=1)
# gold = gold.drop(['anticipation','trust','surprise','pessimism'],axis=1)

In [ ]:
plt.hist(trainData['Tweet'].str.split().apply(len))

In [5]:
from keras.models import Model
from keras.models import Sequential
from keras.layers import Input, Dense,TimeDistributed, Embedding, SpatialDropout1D,Concatenate, concatenate,MaxoutDense,LSTM,GaussianNoise,Average,Flatten
from keras.layers import GRU, Bidirectional, GlobalAveragePooling1D, GlobalMaxPooling1D, Conv1D, Dropout,Activation,MaxPooling1D,Merge
from keras.preprocessing import text, sequence

In [6]:
max_features = 20000
maxlen = 50
embed_size = 300

In [ ]:
# trainData['Tweet'] = trainData['Tweet'].map(lambda com : clean(com))
# devData['Tweet'] = devData['Tweet'].map(lambda com : clean(com))
# testData['Tweet'] = testData['Tweet'].map(lambda com : clean(com))

In [14]:
tokenizer = text.Tokenizer(num_words=max_features,filters='')
tokenizer.fit_on_texts(trainData['Tweet'].tolist()+devData['Tweet'].tolist()+testData['Tweet'].tolist())
X_train = tokenizer.texts_to_sequences(trainData['Tweet'].tolist())
X_val = tokenizer.texts_to_sequences(devData['Tweet'].tolist())
X_test = tokenizer.texts_to_sequences(testData['Tweet'].tolist())
x_train = sequence.pad_sequences(X_train, maxlen=maxlen)
x_val = sequence.pad_sequences(X_val, maxlen=maxlen)
x_test = sequence.pad_sequences(X_test, maxlen=maxlen)

In [8]:
y_train = trainData.loc[:, trainData.columns != 'Tweet'].as_matrix()
y_val = devData.loc[:, devData.columns != 'Tweet'].as_matrix()
y_test = testData.loc[:, testData.columns != 'Tweet'].as_matrix()
y_gold = gold.loc[:, testData.columns != 'Tweet'].as_matrix()

y_test = np.array(y_test.tolist())
# y_test = y_test[:,10]

y_train = np.array(y_train.tolist())
# y_train = y_train[:,10]

y_val = np.array(y_val.tolist())
# y_val = y_val[:,10]

y_gold = np.array(y_gold.tolist())
# y_gold = y_gold[:,10]

In [15]:
tokenizer.word_index[',']

8

In [ ]:
from gensim.models import KeyedVectors

In [ ]:
EMBEDDING_FILE = '/media/prabod/37727493640E51BE/GoogleNews-vectors-negative300.bin.gz'

In [ ]:
EMBEDDING_FILE = 'embeddings/w2v.vec'

In [ ]:
EMBEDDING_FILE = '/media/prabod/37727493640E51BE/word2vec_twitter_model/word2vec_twitter_model.bin'

In [ ]:
EMBEDDING_FILE = 'glove.twitter.27B.200d.txt'

In [ ]:
EMBEDDING_FILE = 'wiki-news-300d-1M.vec'

In [16]:
EMBEDDING_FILE = 'datastories.twitter.300d.txt'

In [ ]:
from word2vecReader import Word2Vec

In [ ]:
embeddings_index=Word2Vec.load_word2vec_format(EMBEDDING_FILE, binary=False)

In [ ]:
embeddings_index['dog'].shape

In [17]:
def get_coefs(word, *arr): return word, np.asarray(arr, dtype='float32')
embeddings_index = dict(get_coefs(*o.rstrip().rsplit(' ')) for o in open(EMBEDDING_FILE))

In [ ]:
word_index = tokenizer.word_index
nb_words = max(max_features, len(word_index))
embedding_matrix = np.zeros((nb_words, embed_size))
without_embed = []
n=0
for word, i in word_index.items():
    if i >= max_features: break
    try:
        embedding_vector = embeddings_index[word]
        if embedding_vector is not None: 
            embedding_matrix[i] = embedding_vector
    except:
            n+=1
            without_embed.append(word)
print "Words without embeddings %d" % n

In [18]:
word_index = tokenizer.word_index
nb_words = max(max_features, len(word_index))
embedding_matrix = np.zeros((nb_words, embed_size))
without_embed = []
n=0
for word, i in word_index.items():
    if i >= max_features: break
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None: 
        embedding_matrix[i] = embedding_vector
    else:
        n+=1
        without_embed.append(word)
print "Words without embeddings %d" % n

Words without embeddings 566


Run the below to save the matrix as a pickle

In [19]:
import cPickle as pickle
with open('preprocessed/embedding_matrix_300_punc.p','wb') as f:
    pickle.dump(embedding_matrix,f)

In [ ]:
print without_embed
# print word_index

In [ ]:
embedding_matrix

In [9]:
import matplotlib.pyplot as plt
%matplotlib inline
ass = trainData.drop('Tweet',1).sum()
ass = pd.DataFrame(ass,columns=['s'])
ass['w'] = (ass['s']/ass['s'].sum())
weights = ass['w'].tolist()
print weights

[0.1585244267198405, 0.060942173479561317, 0.16213858424725822, 0.07739282153539381, 0.1543494516450648, 0.04361914257228315, 0.12362911266201396, 0.04953888334995015, 0.1251246261216351, 0.022495014955134597, 0.022245762711864406]


In [ ]:
1/np.round(-np.log(weights)*1.)

In [10]:
import keras.backend as K
def weighted_multitask_loss(y_true, y_pred):
    # Avoid divide by 0
    W = np.array(weights,dtype=np.float32)
    y_pred = K.clip(y_pred, K.epsilon(), 1 - K.epsilon())
    # Multi-task loss
    return K.mean(K.sum((- y_true * -K.log(W)*K.log(y_pred) - (1.0 - y_true) *K.log(1.0 - y_pred)), axis=1))


In [ ]:
import keras.backend as K
def fbased(y_true, y_pred):
    y_pred = K.clip(y_pred, K.epsilon(), 1 - K.epsilon())
    return K.mean(1 - (2 * y_pred * y_true)/(K.sum(y_true) + K.sum(y_pred)))

In [11]:
def binary_crossentropy_weighted(y_true, y_pred):
    class_weights = 2
    y_pred = K.clip(y_pred, K.epsilon(), 1.0 - K.epsilon())
    tmp =class_weights*((-y_true * K.log(y_pred))) -(1.0 - y_true) * K.log(1.0 - y_pred)
    loss = K.mean(tmp,axis=-1)
    return loss

In [ ]:
def identity_loss(y_true, y_pred):

    return K.mean(y_pred - 0 * y_true)

In [ ]:
import tensorflow as tf
import keras.backend.tensorflow_backend as tfb

POS_WEIGHT = 2  # multiplier for positive targets, needs to be tuned

def weighted_binary_crossentropy(target, output):
    """
    Weighted binary crossentropy between an output tensor 
    and a target tensor. POS_WEIGHT is used as a multiplier 
    for the positive targets.

    Combination of the following functions:
    * keras.losses.binary_crossentropy
    * keras.backend.tensorflow_backend.binary_crossentropy
    * tf.nn.weighted_cross_entropy_with_logits
    """
    # transform back to logits
    _epsilon = tfb._to_tensor(tfb.epsilon(), output.dtype.base_dtype)
    output = tf.clip_by_value(output, _epsilon, 1 - _epsilon)
    output = tf.log(output / (1 - output))
    # compute weighted loss
    loss = tf.nn.weighted_cross_entropy_with_logits(targets=target,
                                                    logits=output,
                                                    pos_weight=POS_WEIGHT)
    return tf.reduce_mean(loss, axis=-1)

In [ ]:
import tensorflow as tf
import keras.backend.tensorflow_backend as tfb

POS_WEIGHT = 2  # multiplier for positive targets, needs to be tuned

def loss1(target, output):
    _epsilon = tfb._to_tensor(tfb.epsilon(), output.dtype.base_dtype)
    output = tf.clip_by_value(output, _epsilon, 1 - _epsilon)
    output = tf.log(output / (1 - output))
    return tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=output, labels=target))

In [ ]:
from keras import backend as K, initializers, regularizers, constraints
from keras.engine.topology import Layer
def dot_product(x, kernel):
    """
    Wrapper for dot product operation, in order to be compatible with both
    Theano and Tensorflow
    Args:
        x (): input
        kernel (): weights
    Returns:
    """
    if K.backend() == 'tensorflow':
        return K.squeeze(K.dot(x, K.expand_dims(kernel)), axis=-1)
    else:
        return K.dot(x, kernel)
    

class AttentionWithContext(Layer):
    """
    Attention operation, with a context/query vector, for temporal data.
    Supports Masking.
    Follows the work of Yang et al. [https://www.cs.cmu.edu/~diyiy/docs/naacl16.pdf]
    "Hierarchical Attention Networks for Document Classification"
    by using a context vector to assist the attention
    # Input shape
        3D tensor with shape: `(samples, steps, features)`.
    # Output shape
        2D tensor with shape: `(samples, features)`.
    How to use:
    Just put it on top of an RNN Layer (GRU/LSTM/SimpleRNN) with return_sequences=True.
    The dimensions are inferred based on the output shape of the RNN.
    Note: The layer has been tested with Keras 2.0.6
    Example:
        model.add(LSTM(64, return_sequences=True))
        model.add(AttentionWithContext())
        # next add a Dense layer (for classification/regression) or whatever...
    """

    def __init__(self,
                 W_regularizer=None, u_regularizer=None, b_regularizer=None,
                 W_constraint=None, u_constraint=None, b_constraint=None,
                 bias=True, **kwargs):

        self.supports_masking = True
        self.init = initializers.get('glorot_uniform')

        self.W_regularizer = regularizers.get(W_regularizer)
        self.u_regularizer = regularizers.get(u_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)

        self.W_constraint = constraints.get(W_constraint)
        self.u_constraint = constraints.get(u_constraint)
        self.b_constraint = constraints.get(b_constraint)

        self.bias = bias
        super(AttentionWithContext, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape) == 3

        self.W = self.add_weight((input_shape[-1], input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        if self.bias:
            self.b = self.add_weight((input_shape[-1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)

        self.u = self.add_weight((input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_u'.format(self.name),
                                 regularizer=self.u_regularizer,
                                 constraint=self.u_constraint)

        super(AttentionWithContext, self).build(input_shape)

    def compute_mask(self, input, input_mask=None):
        # do not pass the mask to the next layers
        return None

    def call(self, x, mask=None):
        uit = dot_product(x, self.W)

        if self.bias:
            uit += self.b

        uit = K.tanh(uit)
        ait = dot_product(uit, self.u)

        a = K.exp(ait)

        # apply mask after the exp. will be re-normalized next
        if mask is not None:
            # Cast the mask to floatX to avoid float64 upcasting in theano
            a *= K.cast(mask, K.floatx())

        # in some cases especially in the early stages of training the sum may be almost zero
        # and this results in NaN's. A workaround is to add a very small positive number ε to the sum.
        # a /= K.cast(K.sum(a, axis=1, keepdims=True), K.floatx())
        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())

        a = K.expand_dims(a)
        weighted_input = x * a
        return K.sum(weighted_input, axis=1)

    def compute_output_shape(self, input_shape):
        return input_shape[0], input_shape[-1]


In [ ]:
from attdeep import AttentionWeightedAverage
from attlayer import Attention
from keras.layers import Dropout, SpatialDropout1D, LSTM, Activation,GaussianNoise,Convolution1D,BatchNormalization
from keras.regularizers import L1L2
from keras.optimizers import RMSprop,Adam
from performance import dice_coef,dice_coef_loss,\
jaccard_distance,jaccard_coef_loss,jaccard_coef_loss_1,jaccard_coef,jaccard_coef_int,fmeasure,recall,precision,\
multitask_loss
def gru_model():
    model_input = Input(shape=(maxlen,), dtype='int32')
    embed_reg = L1L2(l2=1e-8)
    embed = Embedding(input_dim=max_features,
                  output_dim=256,
                  mask_zero=True,
                  trainable=False,
                  input_length=maxlen,
                  embeddings_regularizer=embed_reg,
                  name='embedding')
    x = embed(model_input)
#     x = BatchNormalization()(x)
    x = Activation('tanh')(x)
    x = GaussianNoise(0.2)(x)
    embed_drop = SpatialDropout1D(0.4, name='embed_drop')
    x = embed_drop(x)
    lstm_0_output = Bidirectional(LSTM(64, return_sequences=True), name="bi_lstm_0")(x)
    lstm_1_output = Bidirectional(LSTM(64, return_sequences=True), name="bi_lstm_1")(lstm_0_output)
    x = concatenate([lstm_1_output, lstm_0_output, x])
    x = GaussianNoise(0.3)(x)
    x = AttentionWeightedAverage(name='attlayer')(x)
#     x = Attention(name='attlayer')(x)
    x = Dropout(0.2)(x)
    outputs = [Dense(11, activation='sigmoid', name='softmax')(x)]
    model = Model(inputs=[model_input], outputs=outputs, name="DeepMoji")
    model.compile(loss=weighted_multitask_loss,
              optimizer=Adam(clipnorm=1,lr=0.001),
              metrics=['accuracy',fmeasure,recall,precision])
    return model

In [ ]:
from performance import dice_coef,dice_coef_loss,\
jaccard_distance,jaccard_coef_loss,jaccard_coef_loss_1,jaccard_coef,jaccard_coef_int,fmeasure,recall,precision,\
multitask_loss
def get_model():
    sequence_input = Input(shape=(maxlen, ))
    x = Embedding(max_features, embed_size,weights=[embedding_matrix],trainable = True)(sequence_input)
#     x = SpatialDropout1D(0.2)(x)
    x = Bidirectional(GRU(128, return_sequences=True,dropout=0.1,recurrent_dropout=0.1))(x)
    x = Conv1D(64, kernel_size = 2, padding = "valid", kernel_initializer = "glorot_uniform")(x)
    avg_pool = GlobalAveragePooling1D()(x)
    max_pool = GlobalMaxPooling1D()(x)
    x = concatenate([avg_pool, max_pool]) 
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.5)(x)
    preds = Dense(11, activation="sigmoid")(x)
    
    model = Model(inputs=sequence_input, outputs=preds)
    model.compile(loss=weighted_binary_crossentropy,
                  optimizer='adam',
                  metrics=['accuracy',jaccard_coef,dice_coef,fmeasure,recall,precision])

    return model

In [ ]:
from performance import dice_coef,dice_coef_loss,\
jaccard_distance,jaccard_coef_loss,jaccard_coef_loss_1,jaccard_coef,jaccard_coef_int,fmeasure,recall,precision,\
multitask_loss
from attdeep import AttentionWeightedAverage
from attlayer import Attention,AttLayer
from keras.regularizers import l2, L1L2
from keras.optimizers import RMSprop,Adam
def get_model_2():
    model_input = Input(shape=(maxlen,), dtype='int32')
    embed_reg = L1L2(l2=1e-8)
    embed = Embedding(max_features, embed_size,
                      mask_zero=True,
                      weights=[embedding_matrix],
                      trainable=False,
                      name='embedding')
    x = embed(model_input)
#     x = Activation('tanh')(x)
    x = GaussianNoise(0.3)(x)
#     embed_drop = SpatialDropout1D(0.2, name='embed_drop')
#     x = embed_drop(x)
    lstm_0_output = Bidirectional(LSTM(128, return_sequences=True), name="bi_lstm_0")(x)
    lstm_1_output = Bidirectional(LSTM(128, return_sequences=True), name="bi_lstm_1")(lstm_0_output)
    x = concatenate([lstm_1_output, lstm_0_output])
    x = GaussianNoise(0.3)(x)
    x = TimeDistributed(Dense(256))(x)
#     x = AttentionWeightedAverage(name='attlayer')(x)
    x = AttentionWithContext(name='attlayer')(x)
    x = Dropout(0.5)(x)
    outputs = [Dense(11, activation='sigmoid', name='softmax')(x)]
    model = Model(inputs=[model_input], outputs=outputs, name="DeepMoji")
    model.compile(loss=weighted_binary_crossentropy,
              optimizer=Adam(clipnorm=1, lr=0.001),
              metrics=['accuracy',fmeasure,recall,precision])

    return model

In [ ]:
from performance import dice_coef,dice_coef_loss,\
jaccard_distance,jaccard_coef_loss,jaccard_coef_loss_1,jaccard_coef,jaccard_coef_int,fmeasure,recall,precision,\
multitask_loss
from attdeep import AttentionWeightedAverage
from attlayer import Attention
from keras.regularizers import l2, L1L2
from keras.optimizers import RMSprop,Adam
from keras.layers import Dropout, SpatialDropout1D, LSTM, Activation,Convolution1D,MaxPooling1D
def get_model_3():
    filter_sizes = (2,4,5,8)
    dropout_prob = [0.4,0.5]

    graph_in = Input(shape=(maxlen, embed_size))
    convs = []
    avgs = []
    for fsz in filter_sizes:
        conv = Convolution1D(nb_filter=32,
                             filter_length=fsz,
                             border_mode='valid',
                             activation='relu',
                             subsample_length=1)(graph_in)
        pool = MaxPooling1D(pool_length=maxlen-fsz+1)(conv)
        flattenMax = Flatten()(pool)
        convs.append(flattenMax)

    if len(filter_sizes)>1:
        out = Merge(mode='concat')(convs)
    else:
        out = convs[0]

    graph = Model(input=graph_in, output=out, name="graphModel")
    
    model_input = Input(shape=(maxlen,), dtype='int32')
    embed = Embedding(max_features, embed_size,
#                       mask_zero=True,
                      weights=[embedding_matrix],
                      trainable=False,
                      name='embedding')
    x = embed(model_input)
    x = graph(x)
    x = Dense(128)(x)
#     x = MaxPooling1D(pool_length=pool_length)(x)
    x = LSTM(70)(x)
    outputs = [Dense(11, activation='sigmoid', name='softmax')(x)]
    model = Model(inputs=[model_input], outputs=outputs, name="DeepMoji")
    model.compile(loss=weighted_multitask_loss,
              optimizer=Adam(clipnorm=1, lr=0.001),
              metrics=['accuracy',fmeasure,recall,precision])

    return model

In [ ]:
from attdeep import AttentionWeightedAverage
from keras.regularizers import l2, L1L2
from keras.optimizers import RMSprop,Adam
def get_cnn_model_v3():    # added filter
    sequence_input = Input(shape=(maxlen, ))
    x = Embedding(max_features, embed_size,weights=[embedding_matrix],trainable = False)(sequence_input)
    drop = SpatialDropout1D(0.2)(x)
#     x = Conv1D(128, kernel_size = 2, padding = "valid", kernel_initializer = "glorot_uniform")(x)
#     x = GlobalMaxPooling1D()(x)
    d = Dense(128, activation='relu')(x)
    x = concatenate([d,drop])
    x = GaussianNoise(0.3)(x)
#     x = Bidirectional(LSTM(128, return_sequences=True,dropout=0.1,recurrent_dropout=0.1))(x)
    x = Bidirectional(GRU(128, return_sequences=True,dropout=0.1,recurrent_dropout=0.1))(x)
    x = Conv1D(64, kernel_size = 2, padding = "valid", kernel_initializer = "glorot_uniform",kernel_regularizer=L1L2(l2=1e-4))(x)
    avg_pool = GlobalAveragePooling1D()(x)
    max_pool = GlobalMaxPooling1D()(x)
    x = concatenate([avg_pool, max_pool]) 
    x = GaussianNoise(0.3)(x)
#     x = Dense(64, activation='relu',activity_regularizer=L1L2(l2=1e-4))(x)
    x = Dense(128, activation='relu',activity_regularizer=L1L2(l2=1e-4),kernel_regularizer=L1L2(l2=1e-4))(x)
    x = Dropout(0.5)(x)
    x = GaussianNoise(0.3)(x)
#     x = MaxoutDense(11)(x)
    preds = Dense(11, activation="sigmoid",activity_regularizer=L1L2(l2=1e-4),kernel_regularizer=L1L2(l2=1e-4))(x)
    
    model = Model(inputs=sequence_input, outputs=preds)
    model.compile(loss=weighted_multitask_loss,
                  optimizer=Adam(clipnorm=1, lr=0.001),
                  metrics=['acc',fmeasure,recall,precision])
    return model


In [ ]:
from attdeep import AttentionWeightedAverage
from keras.regularizers import l2, L1L2
from attlayer import Attention
from keras.layers import Dropout, SpatialDropout1D, LSTM, Activation,GaussianNoise,Convolution1D
from keras.optimizers import RMSprop,Adam
def model_all():    # added filter
    
    filter_sizes = (2,4,5,8)
    dropout_prob = [0.4,0.5]

    graph_in = Input(shape=(maxlen, embed_size))
    convs = []
    avgs = []
    for fsz in filter_sizes:
        conv = Convolution1D(nb_filter=32,
                             filter_length=fsz,
                             border_mode='valid',
                             activation='relu',
                             subsample_length=1)(graph_in)
        pool = MaxPooling1D(pool_length=maxlen-fsz+1)(conv)
        flattenMax = Flatten()(pool)
        convs.append(flattenMax)

    if len(filter_sizes)>1:
        out = Merge(mode='concat')(convs)
    else:
        out = convs[0]

    graph = Model(input=graph_in, output=out, name="graphModel")
    
    sequence_input = Input(shape=(maxlen, ))
    embedding = Embedding(max_features, embed_size,weights=[embedding_matrix],trainable = False)(sequence_input)
    drop = SpatialDropout1D(0.2)(embedding)
#     d = Dense(128, activation='relu')(embedding)
#     x = concatenate([d,drop])
    g_noise = GaussianNoise(0.3)(embedding)
#     x = Bidirectional(LSTM(128, return_sequences=True,dropout=0.1,recurrent_dropout=0.1))(x)
    bi_lstm = Bidirectional(GRU(128, return_sequences=True,dropout=0.1,recurrent_dropout=0.1))(g_noise)
#     lstm_0_output = Bidirectional(GRU(35, return_sequences=True), name="bi_lstm_0")(g_noise)
#     lstm_1_output = Bidirectional(GRU(35, return_sequences=True), name="bi_lstm_1")(lstm_0_output)
#     bi_lstm = concatenate([lstm_1_output, lstm_0_output,drop])
    conv = Conv1D(64, kernel_size = 2, padding = "valid", kernel_initializer = "glorot_uniform")(bi_lstm)
    avg_pool = GlobalAveragePooling1D()(conv)
    max_pool = GlobalMaxPooling1D()(conv)
    atten = Attention()(bi_lstm)
    time_dist = TimeDistributed(Dense(256))(bi_lstm)
    time_atten = AttentionWithContext()(time_dist)
    conv_set = graph(embedding)
#     conc_all = concatenate([avg_pool, max_pool,atten,time_atten])
    pool = concatenate([avg_pool, max_pool])
    x1 = Dense(11, activation='sigmoid',activity_regularizer=L1L2(l2=1e-4),kernel_regularizer=L1L2(l2=1e-4))(pool)
    x2 = Dense(11, activation='sigmoid',activity_regularizer=L1L2(l2=1e-4),kernel_regularizer=L1L2(l2=1e-4))(atten)
#     x3 = Dense(11, activation='sigmoid',activity_regularizer=L1L2(l2=1e-4),kernel_regularizer=L1L2(l2=1e-4))(time_dist)
    x4 = Dense(11, activation='sigmoid',activity_regularizer=L1L2(l2=1e-4),kernel_regularizer=L1L2(l2=1e-4))(time_atten)
    x5 = Dense(11, activation='sigmoid',activity_regularizer=L1L2(l2=1e-4),kernel_regularizer=L1L2(l2=1e-4))(conv_set)
    
    conc_all = concatenate([x1,x2,x4,x5])
    x = GaussianNoise(0.3)(conc_all)
#     x = Dense(64, activation='relu',activity_regularizer=L1L2(l2=1e-4))(x)
#     x = Dropout(0.5)(x)
#     x = GaussianNoise(0.3)(x)
#     x = MaxoutDense(11)(x)
    preds = Dense(11, activation="sigmoid",activity_regularizer=L1L2(l2=1e-4),kernel_regularizer=L1L2(l2=1e-4))(x)
    
    model = Model(inputs=sequence_input, outputs=preds)
    model.compile(loss=weighted_multitask_loss,
                  optimizer=Adam(lr=0.001),
                  metrics=['acc',fmeasure,recall,precision])
    return model


In [ ]:
from attdeep import AttentionWeightedAverage
from keras.regularizers import l2, L1L2
from keras.optimizers import RMSprop,Adam
def get_cnn_model_dual_embed():    # added filter
    sequence_input = Input(shape=(maxlen, ))
    embed_reg = L1L2(l2=1e-6)
    x1 = Embedding(max_features, embed_size,weights=[embedding_matrix],trainable = False)(sequence_input)
    x2 = Embedding(max_features, embed_size,weights=[embedding_matrix_dup],trainable = True,embeddings_regularizer=embed_reg)(sequence_input)
    x = Concatenate(axis=1)([x1,x2])
    x = GaussianNoise(0.2)(x)
    embed_drop = SpatialDropout1D(0.2, name='embed_drop')
    x = embed_drop(x)
    lstm_0_output = Bidirectional(LSTM(70, return_sequences=True), name="bi_lstm_0")(x)
    lstm_1_output = Bidirectional(LSTM(70, return_sequences=True), name="bi_lstm_1")(lstm_0_output)
    x = concatenate([lstm_1_output, lstm_0_output, x])
    x = GaussianNoise(0.2)(x)
    x = AttentionWeightedAverage(name='attlayer')(x)
#     x = Attention(name='attlayer')(x)
    x = Dropout(0.2)(x)
    outputs = [Dense(11, activation='sigmoid', name='softmax')(x)]
    model = Model(inputs=sequence_input, outputs=outputs, name="DeepMoji")
    model.compile(loss=weighted_multitask_loss,
              optimizer=Adam(clipnorm=1, lr=0.001),
              metrics=['accuracy',fmeasure,recall,precision])
    return model


In [ ]:
from attdeep import AttentionWeightedAverage
from keras.regularizers import l2, L1L2
from keras.layers import Dropout, SpatialDropout1D, LSTM, Activation,GaussianNoise,Convolution1D
def get_cnn_model_v4():    # added filter
#     embedded_sequences = Bidirectional(GRU(128, return_sequences=True,dropout=0.1,recurrent_dropout=0.1))(embedded_sequences)
    filter_sizes = (2,4,5,8)
    dropout_prob = [0.4,0.5]

    graph_in = Input(shape=(maxlen, embed_size))
    convs = []
    avgs = []
    for fsz in filter_sizes:
        conv = Convolution1D(nb_filter=32,
                             filter_length=fsz,
                             border_mode='valid',
                             activation='relu',
                             subsample_length=1)(graph_in)
        pool = MaxPooling1D(pool_length=maxlen-fsz+1)(conv)
        flattenMax = Flatten()(pool)
        convs.append(flattenMax)

    if len(filter_sizes)>1:
        out = Merge(mode='concat')(convs)
    else:
        out = convs[0]

    graph = Model(input=graph_in, output=out, name="graphModel")

    model = Sequential()
    model.add(Embedding(input_dim=max_features, #size of vocabulary
                     output_dim = embed_size,
                     input_length = maxlen,
#                      weights=[embedding_matrix],
                     trainable=False))
    model.add(Dropout(dropout_prob[0]))
    model.add(Bidirectional(GRU(100, return_sequences=True,dropout=0.1,recurrent_dropout=0.1)))
    model.add(graph)
    model.add(Dense(128))
    model.add(Dropout(dropout_prob[1]))
    model.add(Activation('relu'))
    model.add(Dense(11))
    model.add(Activation('sigmoid'))
    model.compile(loss=weighted_multitask_loss,
                  optimizer=Adam(clipnorm=1, lr=0.001),
                  metrics=['acc',fmeasure,recall,precision])
    return model

In [ ]:
model = get_model()

In [ ]:
X_tra, X_val, y_tra, y_val = x_train,x_val, y_train,y_val 

In [ ]:
y_val

In [ ]:
from keras.callbacks import Callback
from sklearn.metrics import roc_auc_score,jaccard_similarity_score
class RocAucEvaluation(Callback):
    def __init__(self, validation_data=(), interval=1):
        super(Callback, self).__init__()

        self.interval = interval
        self.X_val, self.y_val = validation_data
#     def on_batch_end(self, batch, logs={}):
#         y_pred = self.model.predict(self.X_val, verbose=0)
#         score = roc_auc_score(self.y_val, y_pred)
#         jac_score = jaccard_similarity_score(self.y_val, y_pred.round(), normalize=False)
#         logs['roc_auc_val'] = score
#         logs['jac_val'] = jac_score
    
    def on_epoch_end(self, epoch, logs={}):
        if epoch % self.interval == 0:
            y_pred = self.model.predict(self.X_val, verbose=0)
            score = roc_auc_score(self.y_val, y_pred)
            jac_score = jaccard_similarity_score(self.y_val, y_pred.round())
            logs['roc_auc_val'] = score
            logs['jac_val'] = jac_score
            print("\n ROC-AUC - epoch: %d - score: %.6f \n" % (epoch+1, score))
            print("\n JAC-SIM - epoch: %d - score: %.6f \n" % (epoch+1, jac_score))



In [ ]:
from keras.callbacks import TensorBoard, ReduceLROnPlateau, ModelCheckpoint, EarlyStopping
RocAuc = RocAucEvaluation(validation_data=(X_val, y_val), interval=1)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', verbose=1,factor=0.2,patience=3, min_lr=0.0001)
checkpointer = ModelCheckpoint(filepath='models/weights_exp_105_preprocessed.hdf5', monitor='jac_val',verbose=1,mode='max', save_best_only=True)
earlyStoper = EarlyStopping(monitor='jac_val', min_delta=0, patience=7, verbose=1, mode='max')


In [ ]:
from keras.callbacks import TensorBoard, ReduceLROnPlateau, ModelCheckpoint, EarlyStopping
RocAuc = RocAucEvaluation(validation_data=(X_val, y_val), interval=1)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', verbose=1,factor=0.2,patience=3, min_lr=0.0001)
checkpointer = ModelCheckpoint(filepath='models/weights_exp_105_preprocessed.hdf5', monitor='val_loss',verbose=1,mode='min', save_best_only=True)
earlyStoper = EarlyStopping(monitor='val_loss', min_delta=0, patience=5, verbose=1, mode='min')

In [12]:
from performance import dice_coef,dice_coef_loss,\
jaccard_distance,jaccard_coef_loss,jaccard_coef_loss_1,jaccard_coef,jaccard_coef_int,fmeasure,recall,precision,\
multitask_loss

In [ ]:
from collections import Counter
def get_class_weights(y, smooth_factor=0):
    """
    Returns the weights for each class based on the frequencies of the samples
    :param smooth_factor: factor that smooths extremely uneven weights
    :param y: list of true labels (the labels must be hashable)
    :return: dictionary with the weight for each class
    """
    counter = Counter(y)

    if smooth_factor > 0:
        p = max(counter.values()) * smooth_factor
        for k in counter.keys():
            counter[k] += p

    majority = max(counter.values())

    return {cls: float(majority / count) for cls, count in counter.items()}

In [ ]:
class_weights=get_class_weights(y_train.tolist(),0)
print class_weights

In [ ]:
batch_size = 32
epochs = 100

In [ ]:
OUT_FILE='Prediction/E-C_en_pred_exp_105.txt'

In [ ]:
# model = get_cnn_model_dual_embed()
from sklearn.cross_validation import KFold
n_folds = 10
# data, labels, header_info = load_data()
skf = KFold(len(X_tra),10, random_state=None, shuffle=True)

for i, (train, test) in enumerate(skf):
    print "Running Fold", i+1, "/", n_folds
    model = None # Clearing the NN.
    model = get_cnn_model_v3()
    train_and_evaluate_model(model, X_tra[train], y_tra[train], X_tra[test], y_tra[test])

In [ ]:
from keras.models import load_model
import itertools
def train_and_evaluate_model(model, X_tra, y_tra, X_val, y_val):
    hist = model.fit(X_tra, y_tra, batch_size=batch_size,shuffle=False, epochs=epochs,validation_data=(X_val, y_val),
                     callbacks=[RocAuc,earlyStoper,checkpointer,reduce_lr])

    tData = pd.read_table("datasets/2018-E-c-En-test.txt",sep="\t", index_col=0, quoting=csv.QUOTE_NONE)
    model = load_model('models/weights_exp_105_preprocessed.hdf5',custom_objects={'AttentionWeightedAverage': AttentionWeightedAverage,
                                                                                 'AttentionWithContext':AttentionWithContext,
                                                                                 'dice_coef_loss':dice_coef_loss,
                                                                                'dice_coef': dice_coef,
                                                                                'jaccard_distance':jaccard_distance,
                                                                                'fmeasure':fmeasure,
                                                                                'recall':recall,
                                                                                'precision':precision,
                                                                                'jaccard_coef':jaccard_coef,
                                                                                'jaccard_coef_loss':jaccard_coef_loss,
                                                                                'weighted_binary_crossentropy':weighted_binary_crossentropy,
                                                                                'multitask_loss':multitask_loss,
                                                                                'weighted_multitask_loss':weighted_multitask_loss})
    y_pred = model.predict(x_test, batch_size=32)
    boool = y_pred >= 0.5
    boool = boool.astype(int)
    conf_mat = confusion_matrix(y_gold,boool,np.array(['anger','anticipation','disgust','fear','joy','love','optimism','pessimism','sadness','surprise','trust']))
    attr_others=['anger','anticipation','disgust','fear','joy','love','optimism','pessimism','sadness','surprise','trust']
    plot_correlation_matrix(boool,attr_others)
    classes = ['anger','anticipation','disgust','fear','joy','love','optimism','pessimism','sadness','surprise','trust']
    print(jaccard_similarity_score(y_gold, boool))
    print(classification_report(y_gold, boool))
# tData.loc[:,['anger','anticipation','disgust','fear','joy','love','optimism','pessimism','sadness','surprise','trust']] = boool
# tData.to_csv(OUT_FILE,header=True, index=True,sep='\t')

In [ ]:
model = get_cnn_model_v3()
model.summary()
train_and_evaluate_model(model, X_tra, y_tra, X_val, y_val)

In [ ]:
model = get_cnn_model_v3()
model.summary()
train_and_evaluate_model(model, X_tra, y_tra, X_val, y_val)

In [ ]:
pred_data = pd.read_table("Prediction/E-C_en_pred_exp_47.txt",sep="\t", index_col=0, quoting=csv.QUOTE_NONE)
pd.set_option('display.max_colwidth', -1)
gold

In [ ]:
pred_data

In [ ]:
attr_others=['anger','anticipation','disgust','fear','joy','love','optimism','pessimism','sadness','surprise','trust']
plot_correlation_matrix(y_train,attr_others)

In [ ]:
score,acc,f,re,p = model.evaluate(x_test, y_test)

print(score,acc,f,re,p)

In [ ]:
from keras.utils import plot_model
plot_model(model, show_shapes=True,to_file='model_98.png')

In [ ]:
hist.history

In [ ]:
histG = modelG.fit(X_tra, y_tra, batch_size=batch_size, epochs=epochs, validation_data=(X_val, y_val),
                 callbacks=[reduce_lr,earlyStoper,checkpointer,RocAuc])


y_predG = model.predict(x_test, batch_size=1024)
booolG = y_predG >= 0.5
booolG = booolG.astype(int)
testData.loc[:,['anger','anticipation','disgust','fear','joy','love','optimism','pessimism','sadness','surprise','trust']] = boool
testData.to_csv(OUT_FILE,header=True, index=True,sep='\t')

In [ ]:
print(jaccard_similarity_score(y_gold, boool))

In [15]:
import itertools
from sklearn.metrics import roc_auc_score,jaccard_similarity_score
from attdeep import AttentionWeightedAverage
OUT_FILE='Prediction/E-C_en_pred_exp_177.txt'
from keras.models import load_model
modelG = load_model('models/weights_exp_177_preprocessed.hdf5',custom_objects={'AttentionWeightedAverage': AttentionWeightedAverage,
                                                                                # 'Attention':Attention,
#                                                                                 'AttentionM':AttentionM,
#                                                                                 'AttentionMC':AttentionMC,
                                                                                'binary_crossentropy_weighted':binary_crossentropy_weighted,
#                                                                                  'AttentionWithContext':AttentionWithContext,
                                                                                 'dice_coef_loss':dice_coef_loss,
                                                                                'dice_coef': dice_coef,
                                                                                'jaccard_distance':jaccard_distance,
                                                                                'fmeasure':fmeasure,
                                                                                'recall':recall,
                                                                                'precision':precision,
                                                                                'jaccard_coef':jaccard_coef,
                                                                                'jaccard_coef_loss':jaccard_coef_loss,
#                                                                                 'weighted_binary_crossentropy':weighted_binary_crossentropy,
                                                                                'multitask_loss':multitask_loss,
                                                                                'weighted_multitask_loss':weighted_multitask_loss})

testData = pd.read_table("datasets/2018-E-c-En-test.txt",sep="\t", index_col=0, quoting=csv.QUOTE_NONE)
y_pred = modelG.predict(x_test, batch_size=32)
boool = y_pred >= 0.5
boool = boool.astype(int)
classes = ['anger','anticipation','disgust','fear','joy','love','optimism','pessimism','sadness','surprise','trust']
print(classification_report(y_gold, boool, target_names=classes))
print(jaccard_similarity_score(y_gold, boool))
testData.loc[:,['anger','anticipation','disgust','fear','joy','love','optimism','pessimism','sadness','surprise','trust']] = boool
testData.to_csv(OUT_FILE,header=True, index=True,sep='\t')

              precision    recall  f1-score   support

       anger       0.76      0.80      0.78      1101
anticipation       0.38      0.21      0.27       425
     disgust       0.68      0.82      0.74      1099
        fear       0.73      0.75      0.74       485
         joy       0.81      0.88      0.84      1442
        love       0.58      0.68      0.63       516
    optimism       0.63      0.82      0.72      1143
   pessimism       0.42      0.30      0.35       375
     sadness       0.70      0.71      0.71       960
    surprise       0.63      0.10      0.17       170
       trust       0.29      0.03      0.06       153

 avg / total       0.67      0.71      0.68      7869

0.5878965209894943


In [ ]:
y_pred = model.predict(x_test, batch_size=32)
boool = y_pred >= 0.5
boool = boool.astype(int)
classes = ['anger','anticipation','disgust','fear','joy','love','optimism','pessimism','sadness','surprise','trust']
print(classification_report(y_gold, boool, target_names=classes))
print(jaccard_similarity_score(y_gold, boool))
testData.loc[:,['anger','anticipation','disgust','fear','joy','love','optimism','pessimism','sadness','surprise','trust']] = boool
testData.to_csv(OUT_FILE,header=True, index=True,sep='\t')

In [ ]:
p_train=modelG.predict(x_train, batch_size=32)


In [14]:
%run eval/evaluate.py 3 'Prediction/E-C_en_pred_exp_175.txt' 'Prediction/2018-E-c-En-test-gold.txt'

Multi-label accuracy (Jaccard index) between E-C_en_pred_exp_175.txt and 2018-E-c-En-test-gold.txt:	0.5878965209894943
Micro-averaged F1 score between E-C_en_pred_exp_175.txt and 2018-E-c-En-test-gold.txt:	0.7005424278321591
Macro-averaged F1 score between E-C_en_pred_exp_175.txt and 2018-E-c-En-test-gold.txt:	0.5465999633788241


In [ ]:
%run modelCorrelation.py 'Prediction/E-C_en_pred_exp_56.txt' 'Prediction/E-C_en_pred_exp_47.txt'

In [ ]:
plt.plot(y_pred)

In [ ]:
nb_classes = 11

vocab = tokenizer.word_index

In [ ]:
from performance import fmeasure,recall,precision
from attdeep import AttentionWeightedAverage
from performance import multitask_loss, jaccard_distance,dice_coef_loss, dice_coef,jaccard_coef,dice_coef,fmeasure,recall,precision,jaccard_coef_loss

In [ ]:
from keras.models import load_model
model = load_model('models/weights_exp_38_preprocessed.hdf5',custom_objects={'AttentionWeightedAverage': AttentionWeightedAverage,
															'dice_coef_loss':dice_coef_loss,
															'dice_coef': dice_coef,
															'jaccard_distance':jaccard_distance,
															'fmeasure':fmeasure,
															'recall':recall,
															'precision':precision,
															'jaccard_coef':jaccard_coef,
															'jaccard_coef_loss':jaccard_coef_loss,
                                                            'weighted_multitask_loss':weighted_multitask_loss})

In [ ]:
import example_helper
from deepmoji.finetuning import (
    load_benchmark,
    finetune)

In [ ]:
print y_va

In [ ]:
from sklearn.model_selection import train_test_split
X_tr, X_va, y_tr, y_va = train_test_split(x_train, y_train, train_size=0.95, random_state=233)
lmodel.summary()
lmodel, acc = finetune(lmodel, [x_train,x_val,x_test], [y_train, y_val,y_gold], nb_classes,
                      16, method='chain-thaw')
print('Acc: {}'.format(acc))

In [ ]:
y_predG = lmodel.predict(x_test, batch_size=1024)
booolG = y_predG >= 0.5
booolG = booolG.astype(int)
testData.loc[:,['anger','anticipation','disgust','fear','joy','love','optimism','pessimism','sadness','surprise','trust']] = booolG
testData.to_csv(OUT_FILE,header=True, index=True,sep='\t')

In [ ]:
gold.to_csv('Prediction/gold.txt',header=True, index=True,sep='\t')

In [ ]:
# from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt

def confusion_matrix(yt, yp, classes):
    instcount = yt.shape[0]
    n_classes = classes.shape[0]
    mtx = np.zeros((n_classes, 4))
    for i in range(instcount):
        for c in range(n_classes):
            mtx[c,0] += 1 if yt[i,c]==1 and yp[i,c]==1 else 0
            mtx[c,1] += 1 if yt[i,c]==1 and yp[i,c]==0 else 0
            mtx[c,2] += 1 if yt[i,c]==0 and yp[i,c]==0 else 0
            mtx[c,3] += 1 if yt[i,c]==0 and yp[i,c]==1 else 0
    mtx = [[m0/(m0+m1), m1/(m0+m1), m2/(m2+m3), m3/(m2+m3)] for m0,m1,m2,m3 in mtx]
    plt.figure(num=None, figsize=(2, 10), dpi=100, facecolor='w', edgecolor='k')
    plt.imshow(mtx, interpolation='nearest',cmap='Blues')
    plt.title("title")
    tick_marks = np.arange(n_classes)
    plt.xticks(np.arange(4), ['1 - 1','1 - 0','0 - 0','0 - 1'])
    plt.yticks(tick_marks, classes)
    for i, j in itertools.product(range(n_classes), range(4)):
        plt.text(j, i, round(mtx[i][j],2), horizontalalignment="center")

    #plt.tight_layout()
    plt.ylabel('labels')
    plt.xlabel('Predicted')
    plt.show()


In [ ]:
import itertools
# y_pred = model.predict(x_test, batch_size=32)
boool = y_new >= [0.5       , 0.3, 0.5       , 0.4, 0.5       ,
       0.4, 0.5       , 0.4, 0.5       , 0.25      ,
       0.25      ]
# boool = y_new >=0.5
boool = boool.astype(int)
conf_mat = confusion_matrix(y_gold,boool,np.array(['anger','anticipation','disgust','fear','joy','love','optimism','pessimism','sadness','surprise','trust']))
attr_others=['anger','anticipation','disgust','fear','joy','love','optimism','pessimism','sadness','surprise','trust']
plot_correlation_matrix(boool,attr_others)
print(classification_report(y_gold, boool, target_names=classes))
print(jaccard_similarity_score(y_gold, boool))

In [ ]:
%matplotlib inline
pd.scatter_matrix(trainData.loc[:,['anger','anticipation','disgust','fear','joy','love','optimism','pessimism','sadness','surprise','trust']], alpha = 0.2, figsize = (15,15), diagonal = 'kde')
plt.show()

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
ass = trainData.drop('Tweet',1).sum()
ass = pd.DataFrame(ass,columns=['s'])
ass['w'] = 1- (ass['s']/ass['s'].sum())
weights = ass['w'].tolist()

In [ ]:
   
def plot_correlation_matrix(df,col):
    """Takes a pandas dataframe as input"""
    df = pd.DataFrame(df,columns=col)
    fig, ax = plt.subplots(nrows=1, ncols=1,figsize=(10, 10))
    
    cax = ax.matshow(df.corr(),cmap=plt.get_cmap('seismic'))

    ticks = list(range(len(df.columns)))
    ax.set_xticks(ticks)
    ax.set_yticks(ticks)

    ax.set_xticklabels(df.columns, rotation=20, horizontalalignment='left')
    ax.set_yticklabels(df.columns)
    fig.colorbar(cax)
    plt.tight_layout()
    plt.show()

In [ ]:
attr_others=['anger','anticipation','disgust','fear','joy','love','optimism','pessimism','sadness','surprise','trust']
plot_correlation_matrix(y_train,attr_others)

In [ ]:
from keras.models import load_model
mo1 = load_model('models/weights_exp_38_preprocessed.hdf5',custom_objects={'AttentionWeightedAverage': AttentionWeightedAverage,
															'dice_coef_loss':dice_coef_loss,
															'dice_coef': dice_coef,
															'jaccard_distance':jaccard_distance,
															'fmeasure':fmeasure,
															'recall':recall,
															'precision':precision,
															'jaccard_coef':jaccard_coef,
															'jaccard_coef_loss':jaccard_coef_loss,
                                                            'weighted_multitask_loss':weighted_multitask_loss})
mo2 = load_model('models/weights_exp_47_preprocessed.hdf5',custom_objects={'AttentionWeightedAverage': AttentionWeightedAverage,
															'dice_coef_loss':dice_coef_loss,
															'dice_coef': dice_coef,
															'jaccard_distance':jaccard_distance,
															'fmeasure':fmeasure,
															'recall':recall,
															'precision':precision,
															'jaccard_coef':jaccard_coef,
															'jaccard_coef_loss':jaccard_coef_loss,
                                                            'weighted_multitask_loss':weighted_multitask_loss})
mo3 = load_model('models/weights_exp_46_preprocessed.hdf5',custom_objects={'AttentionWeightedAverage': AttentionWeightedAverage,
															'dice_coef_loss':dice_coef_loss,
															'dice_coef': dice_coef,
															'jaccard_distance':jaccard_distance,
															'fmeasure':fmeasure,
															'recall':recall,
															'precision':precision,
															'jaccard_coef':jaccard_coef,
															'jaccard_coef_loss':jaccard_coef_loss,
                                                            'weighted_multitask_loss':weighted_multitask_loss})
models = [mo1,mo2,mo3]

In [ ]:
def ensemble(models, model_input):
    
    outputs = [model.outputs[0] for model in models]
    y = Average()(outputs)
    
    model = Model(model_input, y, name='ensemble')
    
    return model
model_input = Input(shape=(maxlen, ))
ensemble_model = ensemble(models, model_input)

In [ ]:
pred1 = pd.read_table("Prediction/E-C_en_pred_exp_47.txt", sep="\t", index_col=0, quoting=csv.QUOTE_NONE)
pred2 = pd.read_table("Prediction/E-C_en_pred_exp_49.txt", sep="\t", index_col=0, quoting=csv.QUOTE_NONE)
pred3 = pd.read_table("Prediction/E-C_en_pred_exp_34.txt", sep="\t", index_col=0, quoting=csv.QUOTE_NONE)

y_pred1 = pred1.loc[:, pred1.columns != 'Tweet'].as_matrix()
y_pred1 = np.array(y_pred1.tolist())

y_pred2 = pred2.loc[:, pred2.columns != 'Tweet'].as_matrix()
y_pred2 = np.array(y_pred2.tolist())

y_pred3 = pred3.loc[:, pred3.columns != 'Tweet'].as_matrix()
y_pred3 = np.array(y_pred3.tolist())


avg_pred = (y_pred3+ y_pred1+ y_pred2)/3.0

tData = pd.read_table("datasets/2018-E-c-En-test.txt",sep="\t", index_col=0, quoting=csv.QUOTE_NONE)
boool = avg_pred >= 0.5
boool = boool.astype(int)
tData.loc[:,['anger','anticipation','disgust','fear','joy','love','optimism','pessimism','sadness','surprise','trust']] = boool
tData.to_csv('ensemble_5.txt',header=True, index=True,sep='\t')

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
def reliability_curve(y_true, y_score, bins=10, normalize=False):
    """Compute reliability curve

    Reliability curves allow checking if the predicted probabilities of a
    binary classifier are well calibrated. This function returns two arrays
    which encode a mapping from predicted probability to empirical probability.
    For this, the predicted probabilities are partitioned into equally sized
    bins and the mean predicted probability and the mean empirical probabilties
    in the bins are computed. For perfectly calibrated predictions, both
    quantities whould be approximately equal (for sufficiently many test
    samples).

    Note: this implementation is restricted to binary classification.

    Parameters
    ----------

    y_true : array, shape = [n_samples]
        True binary labels (0 or 1).

    y_score : array, shape = [n_samples]
        Target scores, can either be probability estimates of the positive
        class or confidence values. If normalize is False, y_score must be in
        the interval [0, 1]

    bins : int, optional, default=10
        The number of bins into which the y_scores are partitioned.
        Note: n_samples should be considerably larger than bins such that
              there is sufficient data in each bin to get a reliable estimate
              of the reliability

    normalize : bool, optional, default=False
        Whether y_score needs to be normalized into the bin [0, 1]. If True,
        the smallest value in y_score is mapped onto 0 and the largest one
        onto 1.


    Returns
    -------
    y_score_bin_mean : array, shape = [bins]
        The mean predicted y_score in the respective bins.

    empirical_prob_pos : array, shape = [bins]
        The empirical probability (frequency) of the positive class (+1) in the
        respective bins.


    References
    ----------
    .. [1] `Predicting Good Probabilities with Supervised Learning
            <http://machinelearning.wustl.edu/mlpapers/paper_files/icml2005_Niculescu-MizilC05.pdf>`_

    """
    if normalize:  # Normalize scores into bin [0, 1]
        y_score = (y_score - y_score.min()) / (y_score.max() - y_score.min())

    bin_width = 1.0 / bins
    bin_centers = np.linspace(0, 1.0 - bin_width, bins) + bin_width / 2

    y_score_bin_mean = np.empty(bins)
    empirical_prob_pos = np.empty(bins)
    for i, threshold in enumerate(bin_centers):
        # determine all samples where y_score falls into the i-th bin
        bin_idx = np.logical_and(threshold - bin_width / 2 < y_score,
                                 y_score <= threshold + bin_width / 2)
        # Store mean y_score and mean empirical probability of positive class
        y_score_bin_mean[i] = y_score[bin_idx].mean()
        empirical_prob_pos[i] = y_true[bin_idx].mean()
    return y_score_bin_mean, empirical_prob_pos

In [ ]:
import matplotlib.pyplot as plt
reliability_scores = {}
for i in range(11):
    reliability_scores[i]=reliability_curve(y_gold[:,i],y_pred[:,i])
    
for method, (y_score_bin_mean, empirical_prob_pos) in reliability_scores.items():
    plt.figure(method, figsize=(8, 8))
    plt.subplot2grid((3, 1), (0, 0), rowspan=2)
    plt.plot([0.0, 1.0], [0.0, 1.0], 'k', label="Perfect")
    scores_not_nan = np.logical_not(np.isnan(empirical_prob_pos))
    plt.plot(y_score_bin_mean[scores_not_nan],
             empirical_prob_pos[scores_not_nan], label=method)
plt.ylabel("Empirical probability")
plt.legend(loc=0)

# plt.subplot2grid((3, 1), (2, 0))
# for i in range(11):
#     y_score_ = y_pred[:,i]
#     y_score_ = (y_score_ - y_score_.min()) / (y_score_.max() - y_score_.min())
#     plt.hist(y_score_, range=(0, 1), bins=10, label=i,
#              histtype="step", lw=2)
plt.xlabel("Predicted Probability")
plt.ylabel("Count")
plt.legend(loc='upper center', ncol=2)

In [ ]:
from sklearn.isotonic import IsotonicRegression as IR
y_new = np.zeros(y_pred.shape)
# for t in range(11):
#     ir = IR( out_of_bounds = 'clip' )
#     ir.fit( p_train[:,t], y_train[:,t] )
#     y_new[:,t] = ir.transform( y_pred[:,t] )


from sklearn.linear_model import LogisticRegression as LR
for t in range(11):
    lr = LR()                                                       
    lr.fit( p_train[:,t].reshape( -1, 1 ),  y_train[:,t] )     # LR needs X to be 2-dimensional
    y_new[:,t] = lr.predict_proba( y_pred[:,t].reshape( -1, 1 ))[:,1]

In [ ]:
# y_new = np.zeros(y_pred.shape)
y_new

In [ ]:
plt.plot(p_calibrated)

In [ ]:
print p_calibrated
reliability_scoress = {}
for i in range(1):
    reliability_scoress[i]=reliability_curve(y_gold[:,10],p_calibrated)
    
for method, (y_score_bin_mean, empirical_prob_pos) in reliability_scoress.items():
    plt.figure(method, figsize=(8, 8))
    plt.subplot2grid((3, 1), (0, 0), rowspan=2)
    plt.plot([0.0, 1.0], [0.0, 1.0], 'k', label="Perfect")
    scores_not_nan = np.logical_not(np.isnan(empirical_prob_pos))
    plt.plot(y_score_bin_mean[scores_not_nan],
             empirical_prob_pos[scores_not_nan], label=method)